In [ ]:
project_path = "/home/jupyter"
import os
import sys

sys.path.append(project_path)
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
from google.cloud import bigquery

from fintrans_toolbox.src import bq_utils as bq
from fintrans_toolbox.src import table_utils as t


client = bigquery.Client()

In [ ]:
# Select just monthly data and just postal sector to make it easier to work with

query1 = """
SELECT * 
FROM `ons-fintrans-data-prod.fintrans_visa.spend_origin_and_channel`  
WHERE time_period_value = 'Quarter' 
  AND merchant_channel = 'Online' 
  AND cardholder_origin = 'UNITED KINGDOM' 
  AND cardholder_origin_country <> 'All' 
  AND destination_country != 'UNITED KINGDOM'
"""
df_query1 = bq.read_bq_table_sql(client, query1)
df_query1


In [ ]:
# Try to see cardholder_origin_country columns -------------------- ???????????

query2 = '''
SELECT time_period_value, cardholder_origin_country, SUM(spend) AS total 
FROM `ons-fintrans-data-prod.fintrans_visa.spend_origin_and_channel`  
WHERE time_period_value = 'Quarter' 
  AND mcc = 'All' 
  AND mcg = 'BUSINESS TO BUSINESS' 
  AND cardholder_origin = 'UNITED KINGDOM' 
  AND destination_country != 'UNITED KINGDOM'
GROUP BY time_period_value, cardholder_origin_country, mcg 
ORDER BY time_period_value, total DESC
'''
df_query2 = bq.read_bq_table_sql(client, query2)
df_query2


In [ ]:
# Check if B2B issue for Cardholder Origin Country List

query2 = '''
SELECT time_period_value, cardholder_origin_country, SUM(spend) AS total 
FROM `ons-fintrans-data-prod.fintrans_visa.spend_origin_and_channel`  
WHERE time_period_value = 'Quarter' 
  AND mcg IS NOT NULL 
  AND mcg != 'BUSINESS TO BUSINESS' 
  AND mcg != 'All'
  AND cardholder_origin = 'UNITED KINGDOM' 
  AND destination_country != 'UNITED KINGDOM'
GROUP BY time_period_value, cardholder_origin_country 
ORDER BY time_period_value, total DESC
'''
df_query2 = bq.read_bq_table_sql(client, query2)
df_query2


In [ ]:
# Cardholder Origin Country List

query_debug = '''
SELECT DISTINCT cardholder_origin_country
FROM `ons-fintrans-data-prod.fintrans_visa.spend_origin_and_channel`
LIMIT 100
'''
df_debug = bq.read_bq_table_sql(client, query_debug)
df_debug


In [ ]:
mcgquery = '''SELECT time_period_value, mcg, SUM(spend) AS total 
FROM `ons-fintrans-data-prod.fintrans_visa.spend_origin_and_channel` 
where time_period = 'Quarter' 
and merchant_channel = 'Online' 
and cardholder_origin_country = 'POSTAL_AREA' 
and cardholder_origin = 'UNITED KINGDOM' 
and destination_country != 'UNITED KINGDOM' 
GROUP BY mcg, time_period_value 
ORDER BY time_period_value, total DESC'''
df_mcgquery = bq.read_bq_table_sql(client, mcgquery)
df_mcgquery

In [ ]:
mccquery = '''SELECT DISTINCT mcc FROM `ons-fintrans-data-prod.fintrans_visa.spend_origin_and_channel` where merchant_channel = 'Online' and mcg = 'DIRECT MARKETING' and cardholder_origin = 'UNITED KINGDOM' and destination_country != 'UNITED KINGDOM' GROUP BY mcc ORDER BY mcc DESC'''
df_mccquery = bq.read_bq_table_sql(client, mccquery)
df_mccquery

In [ ]:
chlquery = '''SELECT time_period_value, cardholder_location, SUM(spend) AS total 
FROM `ons-fintrans-data-prod.fintrans_visa.spend_origin_and_channel` 
where time_period = 'Quarter' 
and merchant_channel = 'Online' 
and cardholder_origin_country = 'POSTAL_AREA' 
and cardholder_origin = 'UNITED KINGDOM' 
and destination_country != 'UNITED KINGDOM' 
GROUP BY cardholder_location, time_period_value 
ORDER BY time_period_value, total DESC'''
df_chlquery = bq.read_bq_table_sql(client, chlquery)
df_chlquery


In [ ]:
df_chlquery.to_csv('UK_Cardholderlocation_chl.csv')

In [ ]:
dcquery = '''SELECT time_period_value, destination_country, 
SUM(spend) AS total 
FROM `ons-fintrans-data-prod.fintrans_visa.spend_origin_and_channel` 
where time_period = 'Quarter' 
and merchant_channel = 'Online' 
nd cardholder_origin_country = 'POSTAL_AREA' 
and cardholder_origin = 'UNITED KINGDOM' 
and destination_country = 'UNITED KINGDOM' 
GROUP BY cardholder_location, destination_country, time_period_value 
RDER BY time_period_value, total DESC'''
df_dcquery = bq.read_bq_table_sql(client, dcquery)
df_dcquery


In [ ]:
df_dcquery.to_csv('UK_destinationcountry_dc.csv')

In [ ]:
dcqueryintl = '''SELECT time_period_value, destination_country, 
SUM(spend) AS total FROM `ons-fintrans-data-prod.fintrans_visa.spend_origin_and_channel` 
where time_period = 'Quarter' 
and merchant_channel = 'Online' 
and cardholder_origin_country = 'POSTAL_AREA' 
and cardholder_origin = 'UNITED KINGDOM' 
and destination_country != 'UNITED KINGDOM' 
GROUP BY cardholder_location, destination_country, time_period_value 
ORDER BY time_period_value, total DESC'''
df_dcqueryintl = bq.read_bq_table_sql(client, dcqueryintl)
df_dcqueryintl


In [ ]:
df_dcqueryintl.to_csv('Intl_destinationcountry_dc.csv')

In [ ]:
dcqueryindia = '''SELECT time_period_value, SUM(spend) AS total 
FROM `ons-fintrans-data-prod.fintrans_visa.spend_origin_and_channel` 
where time_period = 'Quarter' 
and merchant_channel = 'Online' 
and cardholder_origin_country = 'POSTAL_AREA' 
and cardholder_origin = 'UNITED KINGDOM' 
and destination_country = 'INDIA' 
GROUP BY cardholder_location, time_period_value 
ORDER BY time_period_value, total DESC'''
df_dcqueryindia = bq.read_bq_table_sql(client, dcqueryindia)
df_dcqueryindia 

In [ ]:
df_dcqueryindia.to_csv('India_destinationcountry_dc.csv')

In [ ]:
dcqueryindia1 = '''SELECT time_period_value, mcc, SUM(spend) AS total 
FROM `ons-fintrans-data-prod.fintrans_visa.spend_origin_and_channel` 
where time_period = 'Quarter' 
and merchant_channel = 'Online' 
and cardholder_origin_country = 'POSTAL_AREA' 
and cardholder_origin = 'UNITED KINGDOM' 
and destination_country = 'INDIA' 
and mcc != 'All' 
GROUP BY cardholder_location, time_period_value, mcc 
ORDER BY time_period_value, total DESC'''
df_dcqueryindia1 = bq.read_bq_table_sql(client, dcqueryindia1)
df_dcqueryindia1 

In [ ]:
df_dcqueryindia1.to_csv('India1_destinationcountry_dc.csv')

In [ ]:
dcqueryindia2 = '''SELECT time_period_value, mcc, mcg, cardholder_location, SUM(spend) AS total 
FROM `ons-fintrans-data-prod.fintrans_visa.spend_origin_and_channel` 
where time_period_value = '2024Q2' 
and merchant_channel = 'Online' 
and cardholder_origin_country = 'POSTAL_AREA' 
and cardholder_origin = 'UNITED KINGDOM' 
and destination_country = 'INDIA' 
and mcc = 'AIRLINES' 
GROUP BY cardholder_location, time_period_value, mcc, mcg 
ORDER BY time_period_value, total DESC'''
df_dcqueryindia2 = bq.read_bq_table_sql(client, dcqueryindia2)
df_dcqueryindia2 

In [ ]:
df_dcqueryindia2.to_csv('India2_destinationcountry_dc.csv')

In [ ]:
dcqueryindia3 = '''SELECT time_period_value, mcc, cardholder_location, SUM(spend) AS total 
FROM `ons-fintrans-data-prod.fintrans_visa.spend_origin_and_channel` 
where time_period_value = '2024Q2' 
and merchant_channel = 'Online' 
and cardholder_origin_country = 'All' 
and cardholder_origin = 'UNITED KINGDOM' 
and destination_country = 'INDIA' 
and mcc = 'All' 
ROUP BY cardholder_origin_country, cardholder_location, time_period_value, mcc 
ORDER BY time_period_value, total DESC'''
df_dcqueryindia3 = bq.read_bq_table_sql(client, dcqueryindia3)
df_dcqueryindia3 

In [ ]:
df_dcqueryindia3.to_csv('India3_destinationcountry_dc.csv')

In [ ]:
dcqueryindia4 = '''SELECT time_period_value, mcc, mcg, cardholder_location, SUM(spend) AS total 
FROM `ons-fintrans-data-prod.fintrans_visa.spend_origin_and_channel` 
where time_period_value = '2024Q2' 
and merchant_channel = 'Online' 
and cardholder_origin_country = 'All' 
and cardholder_origin = 'UNITED KINGDOM' 
and destination_country = 'INDIA' 
and mcc = 'AIRLINES' 
GROUP BY cardholder_location, time_period_value, mcc, mcg 
ORDER BY time_period_value, total DESC'''
df_dcqueryindia4 = bq.read_bq_table_sql(client, dcqueryindia4)
df_dcqueryindia4 

In [ ]:
df_dcqueryindia4.to_csv('India4_destinationcountry_dc.csv')